In [ ]:
! wget https://archive.ics.uci.edu/static/public/352/online+retail.zip
! unzip online+retail.zip

In [ ]:
# Products (SKU) of our interets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Read Excel file into a pandas DataFrame
file_path = 'Online Retail.xlsx'  # Replace with your file path
df = pd.read_excel(file_path)

# Display the first few rows of the DataFrame
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
# Select Data from UK
df = df[df['Country'] == 'United Kingdom']
df.shape

(495478, 8)

In [ ]:
# Check Missing Values
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133600
Country             0
dtype: int64

In [ ]:
# Remove missing values
df = df.dropna(subset=['Description', 'CustomerID'])
df.shape

(361878, 8)

In [ ]:
# Remove invalid rows with negative quantity or unit price
df = df[(df['Quantity'] >= 0) & (df['UnitPrice'] >= 0)]
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
# Select completed orders
df['InvoiceNo'].astype(str).str.startswith('5').sum()

np.int64(354345)

In [ ]:
# Get total revenues
df['Revenue'] = df['Quantity'] * df['UnitPrice']

top_20 = df.groupby(['StockCode'])['Revenue'].sum().sort_values(ascending=False).head(20)
top_20.head()

StockCode
23843     168469.60
22423     110990.20
85123A     95013.95
23166      80291.44
85099B     77371.57
Name: Revenue, dtype: float64

In [ ]:
# Save description of top 20 products
top_20_descriptions = df[df['StockCode'].isin(top_20.index)][['StockCode', 'Description']].drop_duplicates()
top_20_descriptions.to_csv('top_20_descriptions.csv', index=False)

In [ ]:
# Remove unnecessary columns
df = df.drop(columns=['Country', 'CustomerID', 'Description'])

# orders of top 20 products
df = df[df['StockCode'].isin(top_20.index)]
df = df.drop(columns=['InvoiceNo', 'Revenue'])
df.shape

(16842, 4)

In [ ]:
# Convert InvoiceDate to datetime
df['InvoiceDate'] = df['InvoiceDate'].dt.strftime('%Y-%m-%d')
df.head()

,StockCode,Quantity,InvoiceDate,UnitPrice
0,85123A,6,2010-12-01,2.55
9,84879,32,2010-12-01,1.69
46,22086,80,2010-12-01,2.55
49,85123A,6,2010-12-01,2.55
66,85123A,6,2010-12-01,2.55


In [ ]:
# Group by StockCode and InvoiceDate and aggregate quantity and unit price
grouped_df = df.groupby(['StockCode', 'InvoiceDate']).agg({'Quantity': 'sum', 'UnitPrice': 'mean'}).reset_index()
print(grouped_df.shape)
grouped_df.head()

(4286, 4)


,StockCode,InvoiceDate,Quantity,UnitPrice
0,21137,2010-12-01,540,3.39
1,21137,2010-12-02,73,3.57
2,21137,2010-12-05,24,3.39
3,21137,2010-12-06,504,3.39
4,21137,2010-12-08,8,3.75


In [53]:
grouped_df.groupby('StockCode').size().sort_values(ascending=False)

StockCode
85123A    304
22423     299
84879     297
85099B    295
47566     283
22197     281
22386     280
22178     277
82484     245
85099F    239
79321     231
22470     226
23298     184
23203     182
21137     166
23284     142
22086     141
22502     108
23166     105
23843       1
dtype: int64

In [54]:
grouped_df.to_csv('datasets/top20_raw.csv', index=False)

In [1]:
import pandas as pd
grouped_df = pd.read_csv('datasets/top20_raw.csv')